In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
0,"About 75,000 more Americans died from COVID-19...",https://www.usatoday.com/story/news/health/202...,"Mon, 12 Oct 2020 15:00:45 GMT",2020-10-12 15:00:45+00:00
6,Eric Trump may not *get* the whole Covid-19 va...,https://www.cnn.com/2020/10/12/politics/eric-t...,"Mon, 12 Oct 2020 14:28:00 GMT",2020-10-12 14:28:00+00:00
8,Maine CDC reports 29 additional COVID-19 cases...,https://www.pressherald.com/2020/10/12/maine-c...,"Mon, 12 Oct 2020 14:24:00 GMT",2020-10-12 14:24:00+00:00
1,Trump takes his Covid misinformation machine b...,https://www.cnn.com/2020/10/12/politics/donald...,"Mon, 12 Oct 2020 12:51:00 GMT",2020-10-12 12:51:00+00:00
5,Eight Persistent COVID-19 Myths and Why People...,https://www.scientificamerican.com/article/eig...,"Mon, 12 Oct 2020 12:06:02 GMT",2020-10-12 12:06:02+00:00
2,When Symptoms of Covid-19 Don’t Go Away The N...,https://www.nytimes.com/2020/10/12/well/live/c...,"Mon, 12 Oct 2020 11:21:00 GMT",2020-10-12 11:21:00+00:00
9,Kids struggle with Covid-19 and its months of ...,https://www.cnn.com/2020/10/12/health/children...,"Mon, 12 Oct 2020 08:10:00 GMT",2020-10-12 08:10:00+00:00
7,COVID round-up: Survey of presidents looks at ...,https://www.insidehighered.com/news/2020/10/12...,"Mon, 12 Oct 2020 07:07:30 GMT",2020-10-12 07:07:30+00:00
4,Governor Cuomo Updates New Yorkers on State's ...,https://www.governor.ny.gov/news/governor-cuom...,"Sun, 11 Oct 2020 17:15:56 GMT",2020-10-11 17:15:56+00:00
3,Coronavirus (COVID-19) Update: Daily Roundup O...,https://www.fda.gov/news-events/press-announce...,"Fri, 09 Oct 2020 19:45:00 GMT",2020-10-09 19:45:00+00:00
